<h1>Import module</h1>

In [1]:
import investpy as inv
import pandas as pd
import sqlite3
import numpy as np
import datetime
import time
import plotly.graph_objects as go
import pandas_ta as ta
import matplotlib
import requests
from random import randint

<h1>Get data manual way</h1>

In [40]:
class SearchObj():
    def __init__(self, id_, name, symbol, country, tag, pair_type, exchange):
        self.id_=id_
        self.name=name
        self.symbol=symbol
        self.country=country
        self.tag=tag
        self.pair_type=pair_type
        self.exchange=exchange
    
    def get_historical_data(self, start_date, end_date):
        """
        start_date format = dd/mm/yyyy
        """

        header = ""
        
        if self.pair_type in ["stocks", "funds", "etfs", "currencies", "certificates"]:
            header = f"{self.symbol} Historical Data"
        elif self.pair_type in ["bonds"]:
            header = f"{self.name} Bond Yield Historical Data"
        elif self.pair_type in ["indices", "commodities", "cryptos", "fxfutures"]:
            header = f"{self.name} Historical Data"

        params = {
            "MIME Type": "application/x-www-form-urlencoded",
            "curr_id": self.id_,
            "smlID": str(randint(1000000, 99999999)),
            "header": header,
            "st_date": start_date,
            "end_date": end_date,
            "interval_sec": "Daily",
            "sort_col": "date",
            "sort_ord": "DESC",
            "action": "historical_data",
            }

        headers = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
        }

        url = "https://www.investing.com/instruments/HistoricalDataAjax"

        req = requests.post(url, headers=headers, data=params)

        hist_data = req.text

        list_of_df = pd.read_html(hist_data, attrs= {"id": "curr_table"})
        merge_hist_df = pd.concat(list_of_df)
        return merge_hist_df

In [26]:
def search_quotes(text, products=None, countries=None, n_results=None):
    
    params = {
        "search_text": text,
        "tab": "quotes",
        "isFilter": True,
        "limit": 270,
        "offset": 0,
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
    }

    url = "https://www.investing.com/search/service/SearchInnerPage"

    search_results = list()
    total_results = None
    user_limit = True if n_results is not None else False

    while True:
        req = requests.post(url, headers=headers, data=params)

        if req.status_code != 200:
            raise ConnectionError(
                f"ERR#0015: error {req.status_code}, try again later."
            )

        data = req.json()

        if data["total"]["quotes"] == 0:
            raise RuntimeError(
                "ERR#0093: no results found on Investing.com for the introduced text."
            )

        if total_results is None:
            total_results = data["total"]["quotes"]

        if n_results is None:
            n_results = data["total"]["quotes"]

        for quote in data["quotes"]:
            country, pair_type = quote["flag"], quote["pair_type"]

            if countries is not None:
                if quote["flag"] not in countries:
                    continue

            if products is not None:
                if quote["pair_type"] not in products:
                    continue

        for quote in data["quotes"]:

            if countries is not None:
                if quote["flag"] not in countries:
                    continue

            if products is not None:
                if quote["pair_type"] not in products:
                    continue

            country = None
            
            country, pair_type = quote["flag"], quote["pair_type"]

            search_obj = SearchObj(
                    id_=quote["pairId"],
                    name=quote["name"],
                    symbol=quote["symbol"],
                    country=quote["flag"],
                    tag=quote["link"],
                    pair_type=pair_type,
                    exchange=quote["exchange"]
                )

            if n_results == 1 and user_limit:
                return search_obj

            if search_obj not in search_results:
                search_results.append(search_obj)

        params["offset"] += 270

        if (
            len(search_results) >= n_results
            or len(search_results) >= total_results
            or params["offset"] >= total_results
        ):
            break

    if len(search_results) < 1:
        raise RuntimeError(
            "ERR#0093: no results found on Investing.com for the introduced query."
        )
    
    return search_results[:n_results]

In [44]:
search_result = search_quotes(text="0P0001J9PO", n_results=1)

In [45]:
historical_data = search_result.get_historical_data("01/01/2020", "01/01/2022")

In [47]:
historical_data

,Date,Price,Open,High,Low,Change %
0,"Dec 31, 2021",17.588,17.588,17.588,17.588,1.68%
1,"Dec 30, 2021",17.297,17.297,17.297,17.297,-3.64%
2,"Dec 24, 2021",17.951,17.951,17.951,17.951,0.45%
3,"Dec 23, 2021",17.871,17.871,17.871,17.871,-0.41%
4,"Dec 22, 2021",17.945,17.945,17.945,17.945,1.53%
...,...,...,...,...,...,...
257,"Dec 14, 2020",18.716,18.716,18.716,18.716,0.00%
258,"Dec 11, 2020",18.716,18.716,18.716,18.716,1.38%
259,"Dec 10, 2020",18.462,18.462,18.462,18.462,-1.83%
260,"Dec 09, 2020",18.806,18.806,18.806,18.806,1.37%


In [27]:
search_results = search_quotes(text="AAPL", n_results=10)
my_list = list()
for result in search_results:
    data = {
        result.name:
        {
            "symbol": result.symbol,
            "country": result.country,
            "type": result.pair_type
        }
    }
    my_list.append(data)
my_list

[{'Apple Inc': {'symbol': 'AAPL', 'country': 'USA', 'type': 'equities'}},
 {'AA Plus Tradelink Ltd': {'symbol': 'AAPL',
   'country': 'India',
   'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Germany', 'type': 'equities'}},
 {'Apple Inc DRC': {'symbol': 'AAPL',
   'country': 'Argentina',
   'type': 'equities'}},
 {'Apple Inc CDR': {'symbol': 'AAPL',
   'country': 'Canada',
   'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Peru', 'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Ukraine', 'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Colombia', 'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Mexico', 'type': 'equities'}},
 {'Apple Inc': {'symbol': 'AAPL', 'country': 'Chile', 'type': 'equities'}}]

In [34]:
for result in search_results:
    print(vars(result))

{'id_': 6408, 'name': 'Apple Inc', 'symbol': 'AAPL', 'country': 'USA', 'tag': '/equities/apple-computer-inc', 'pair_type': 'equities', 'exchange': 'NASDAQ'}
{'id_': 1179023, 'name': 'AA Plus Tradelink Ltd', 'symbol': 'AAPL', 'country': 'India', 'tag': '/equities/aa-plus-tradelink', 'pair_type': 'equities', 'exchange': 'BSE'}
{'id_': 963008, 'name': 'Apple Inc', 'symbol': 'AAPL', 'country': 'Germany', 'tag': '/equities/apple-computer-inc?cid=963008', 'pair_type': 'equities', 'exchange': 'Xetra'}
{'id_': 30128, 'name': 'Apple Inc DRC', 'symbol': 'AAPL', 'country': 'Argentina', 'tag': '/equities/apple-inc.', 'pair_type': 'equities', 'exchange': 'Buenos Aires'}
{'id_': 1178723, 'name': 'Apple Inc CDR', 'symbol': 'AAPL', 'country': 'Canada', 'tag': '/equities/apple-drc', 'pair_type': 'equities', 'exchange': 'NEO'}
{'id_': 1162254, 'name': 'Apple Inc', 'symbol': 'AAPL', 'country': 'Peru', 'tag': '/equities/apple-computer-inc?cid=1162254', 'pair_type': 'equities', 'exchange': 'Lima'}
{'id_': 

<h1>Get Data with investpy</h1>

In [ ]:
!pip install --upgrade git+https://github.com/alvarobartt/investpy.git@master

In [ ]:
search_result = inv.search_quotes(text='0P00018JE1', products=['funds'],n_results=1)
# search_result.retrieve_information()
search_result_hist_data_df = search_result.retrieve_historical_data('20/09/2020', '15/02/2022')

In [ ]:
df = search_result_hist_data_df

<h1>Save data to database</h1>

In [43]:
historical_data

[             Date   Price    Open    High     Low Change %
 0    Dec 31, 2021  17.588  17.588  17.588  17.588    1.68%
 1    Dec 30, 2021  17.297  17.297  17.297  17.297   -3.64%
 2    Dec 24, 2021  17.951  17.951  17.951  17.951    0.45%
 3    Dec 23, 2021  17.871  17.871  17.871  17.871   -0.41%
 4    Dec 22, 2021  17.945  17.945  17.945  17.945    1.53%
 ..            ...     ...     ...     ...     ...      ...
 257  Dec 14, 2020  18.716  18.716  18.716  18.716    0.00%
 258  Dec 11, 2020  18.716  18.716  18.716  18.716    1.38%
 259  Dec 10, 2020  18.462  18.462  18.462  18.462   -1.83%
 260  Dec 09, 2020  18.806  18.806  18.806  18.806    1.37%
 261  Dec 08, 2020  18.551  18.551  18.551  18.551    1.20%
 
 [262 rows x 6 columns]]

In [70]:
df['Date'] = pd.to_datetime(df['Date'])

In [71]:
historical_data

,Date,Price,Open,High,Low,Change %
0,2021-12-31,17.588,17.588,17.588,17.588,1.68%
1,2021-12-30,17.297,17.297,17.297,17.297,-3.64%
2,2021-12-24,17.951,17.951,17.951,17.951,0.45%
3,2021-12-23,17.871,17.871,17.871,17.871,-0.41%
4,2021-12-22,17.945,17.945,17.945,17.945,1.53%
...,...,...,...,...,...,...
257,2020-12-14,18.716,18.716,18.716,18.716,0.00%
258,2020-12-11,18.716,18.716,18.716,18.716,1.38%
259,2020-12-10,18.462,18.462,18.462,18.462,-1.83%
260,2020-12-09,18.806,18.806,18.806,18.806,1.37%


<h1>Analysis</h1>

In [48]:
df = historical_data

In [57]:
"""period = 5
roll_max = df['Price'].rolling(period).max()
daily_drawdown = df['Price']/roll_max - 1
maximum_drawdown = daily_drawdown.rolling(window=period).min()"""

In [76]:
period = 2
max = df['Price'].rolling(window=period).max()
min = df['Price'].rolling(window=period).min()
daily_max_dd = (min-max)/max

In [79]:
period = 6
max = df['Price'].rolling(window=period).max()
min = df['Price'].rolling(window=period).min()
weekly_max_dd = (min-max)/max

In [85]:
daily_max_dd.min()

-0.059248120300751814

In [83]:
daily_max_dd.std()

0.010201540725619563

In [86]:
weekly_max_dd.min()

-0.13262951141664606

In [82]:
weekly_max_dd.std()

0.02120521918833805

In [93]:
fig = go.Figure(data=[ 
    go.Scatter(
        x=df['Date'], 
        y=daily_max_dd,
        line=dict(
            color='green', 
            width=1)
        ),
    go.Scatter(
        x=df['Date'], 
        y=weekly_max_dd,
        line=dict(
            color='red', 
            width=1)
        )
    ])

fig.add_hline(y=daily_max_dd.std())

fig.show()                   

In [94]:
period = 2
max = df['Price'].rolling(window=period).max()
min = df['Price'].rolling(window=period).min()
daily_max_rw = (max-min)/min

In [95]:
period = 5
max = df['Price'].rolling(window=period).max()
min = df['Price'].rolling(window=period).min()
weekly_max_rw = (max-min)/min

In [99]:
fig = go.Figure(data=[ 
    go.Scatter(
        x=df['Date'], 
        y=daily_max_rw,
        line=dict(
            color='green', 
            width=1)
        ),
    go.Scatter(
        x=df['Date'], 
        y=weekly_max_rw,
        line=dict(
            color='red', 
            width=1)
        )
    ])

fig.show()  

In [ ]:
df['MA60'] = df['Close'].rolling(window=60).mean()
df['MA20'] = df['Close'].rolling(window=20).mean()
previous_20 = df['MA20'].shift(1)
previous_60 = df['MA60'].shift(1)
crossing_down = (((df['MA20'] <= df['MA60']) & (previous_20 >= previous_60)))
crossing_up = (((df['MA20'] > df['MA60']) & (previous_20 < previous_60)))
df['crossing_down'] = crossing_down
df['crossing_up'] = crossing_up

In [ ]:
# pandas condition
# df.loc[df['MA20'] < df['MA60'], 'ema60<=200'] = 'True'

In [ ]:
def myplot(df):
    fig = go.Figure(data=[  go.Candlestick( x=df.index,
                                        open=df['Open'],
                                        high=df['High'],
                                        low=df['Low'],
                                        close=df['Close']),
                        go.Scatter( x=df.index, 
                                    y=df['MA60'],
                                    line=dict(
                                        color='red', 
                                        width=1)
                                    ),
                        go.Scatter( x=df.index, 
                                    y=df['MA20'],
                                    line=dict(
                                        color='green', 
                                        width=1)
                                    ),
                        go.Scatter( x=df.loc[df['crossing_down'] == True].index,
                                    y=df.loc[df['crossing_down'] == True]['MA20'],
                                    mode = 'markers',
                                    marker=dict(line=dict(color='black', width = 2),
                                    symbol = 'diamond',
                                    size = 8,
                                    color = 'rgba(255, 255, 0, 0.6)'),
                                    name = 'sell'
                                    ),
                        go.Scatter( x=df.loc[df['crossing_up'] == True].index,
                                    y=df.loc[df['crossing_up'] == True]['MA20'],
                                    mode = 'markers',
                                    marker=dict(line=dict(color='black', width = 2),
                                    symbol = 'diamond',
                                    size = 8,
                                    color = 'green'),
                                    name = 'buy'
                                    ),
                    ],
                    layout=go.Layout(
                        yaxis=dict( autorange = True,
                                    fixedrange= False)
                    )
                )
    fig.show()

In [ ]:
myplot(df)

<h1>Create Database</h1>

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Exception as e:
        print(e)
    return conn

In [ ]:
conn = create_connection('investing_stock_data.db')

<h1>Scrape Data</h1>

In [ ]:
def get_fund_data(country=None):
    try:
        # Scraping
        funds_df = inv.funds.get_funds(country)
        # Parsing
        funds_reorder = funds_df[['symbol', 'name', 'country', 'issuer', 'isin', 'asset_class', 'currency', 'underlying']]
        return funds_reorder
    except Exception as e:
        print(e)

In [ ]:
def get_fund_hist_price(name, country, from_date, to_date):
    try:
        fund_hist_prices_df = inv.get_fund_historical_data(name=name, country=country, from_date=from_date, to_date=to_date)
        return fund_hist_prices_df
    except Exception as e:
        print(e)

<h1>Create Table</h1>

In [ ]:
def sql_query(con, sql_query):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = con.cursor()
        c.execute(sql_query)
    except Exception as e:
        print(e)

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
# con.close()

In [ ]:
# Create table
fundList_table_sql = """CREATE TABLE IF NOT EXISTS 'fundList' (
                        'symbol'    TEXT PRIMARY KEY,
                        'name'      TEXT UNIQUE NOT NULL,
                        'country'   TEXT NOT NULL,
                        'issuer'    TEXT,
                        'isin'      TEXT,
                        'asset_class'   TEXT,
                        'currency'      TEXT,
                        'underlying'    TEXT
                        )"""

fundList_index_sql = """CREATE UNIQUE INDEX IF NOT EXISTS 'idx_symbol'
                        ON 'fundList' ('symbol')"""


# Create table
fundHistPrice_table_sql = """CREATE TABLE IF NOT EXISTS 'fundHistPrice' (
                        'symbol'    TEXT NOT NULL,
                        date        TEXT NOT NULL,
                        open        REAL,
                        high        REAL,
                        low         REAL,
                        close       REAL NOT NULL,
                        currency    TEXT NOT NULL,
                        PRIMARY KEY (symbol, date)
                        FOREIGN KEY ('symbol')
                            REFERENCES 'fundList' ('symbol')
                        )"""

fundHistPrice_index_sql = """CREATE UNIQUE INDEX IF NOT EXISTS 'idx_symbol_date'
                            ON 'fundHistPrice' ('symbol', 'date')"""

sql_query(conn, fundList_table_sql)
sql_query(conn, fundList_index_sql)
sql_query(conn, fundHistPrice_table_sql)
sql_query(conn, fundHistPrice_index_sql)


<h1>Add Data</h1>

In [ ]:
# df to list of tuple for executemany()
fund_data_df = get_fund_data()

In [ ]:
data = list(fund_data_df.to_records(index=False))

In [ ]:
def add_hist_data_sql(data):
    sql_query = """INSERT OR IGNORE INTO 'fundList' ('symbol', 'name', 'country', 'issuer', 'isin', 'asset_class', 'currency', 'underlying')
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
    try:
        c = conn.cursor()
        c.executemany(sql_query, data)
        conn.commit()
    except Exception as e:
        print(e)

In [ ]:
add_hist_data_sql(data)

<h1>Check</h1>

In [ ]:
def read_sql_to_df(table_name, conn):
    sql_query = pd.read_sql_query ('''
                                SELECT * FROM {}
                                '''.format(table_name), conn)
    try:
        df = pd.DataFrame(sql_query)
        return df
    except Exception as e:
        print(e)

In [ ]:
read_sql_to_df(table_name='fundList', conn=conn)

In [ ]:
a = conn.execute("""
                SELECT name, country
                FROM fundList
                WHERE country IN ('thailand')
                """)

# 'ireland', 'luxembourg' ,'thailand', 'united states'
data2 = a.fetchall()


In [ ]:
def add_fund_hist_price_data_to_sql(fund_name, country, from_date, to_date):
    sql_query = """INSERT OR IGNORE 
                    INTO 'fundHistPrice' ('symbol', 'date', 'open', 'high', 'low', 'close', 'currency')
                    VALUES (?, ?, ?, ?, ?, ?, ?)"""

    fund_hist_prices_df = inv.get_fund_historical_data(fund=fund_name, country=country, from_date=from_date, to_date=to_date)
    fund_hist_prices_df['Symbol'] = fund_name
    fund_hist_prices_resetIndex_df = fund_hist_prices_df.reset_index(level=['Date'])
    fund_hist_prices_resetIndex_reorder_df = fund_hist_prices_resetIndex_df[['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Currency']]
    fund_hist_prices_resetIndex_reorder_df['Date'] = fund_hist_prices_resetIndex_reorder_df['Date'].astype(str)
    fund_price_listoftuples = [tuple(i) for i in fund_hist_prices_resetIndex_reorder_df.to_numpy()]
    fund_price_listoftuples

    try:
        c = conn.cursor()
        c.executemany(sql_query, fund_price_listoftuples)
    except Exception as e:
        print(e)

In [ ]:
for i in data2[:6]:
    add_fund_hist_price_data_to_sql(i[0], i[1], from_date='01/01/1900', to_date='10/01/2022')
    print(i[0] + ' done')
    time.sleep(60)

In [ ]:
add_fund_hist_price_data_to_sql('Baillie Gifford Worldwide Long Term Global Growth Fund Class B Usd Acc ', 'ireland', from_date='01/01/1900', to_date='10/01/2022')

In [ ]:
sql_query = pd.read_sql_query ('''
                               SELECT * FROM 'fundHistPrice'
                               ''', conn)

df = pd.DataFrame(sql_query)
df

In [ ]:
sql_query = pd.read_sql_query ('''
                               SELECT DISTINCT symbol 
                               FROM 'fundHistPrice'
                               ''', conn)

df = pd.DataFrame(sql_query)
df

In [ ]:
sql_query = pd.read_sql_query ('''
                               SELECT * FROM 'fundHistPrice'
                               WHERE symbol='Principal Global Equity Fund A'
                               ''', conn)

df = pd.DataFrame(sql_query)
df

In [ ]:
trace1 = {
    'x': df.index,
    'open': df.Open,
    'close': df.Close,
    'high': df.High,
    'low': df.Low,
    'type': 'candlestick',
    'name': 'MSFT',
    'showlegend': True
}